In [1]:
import torch
import torch.nn.functional as F 
import matplotlib.pyplot as plt # For making figures
%matplotlib inline

In [3]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-02-04 16:25:18--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: 'names.txt.2'

names.txt.2         100%[===================>] 222.80K  --.-KB/s    in 0.05s   

2024-02-04 16:25:19 (4.78 MB/s) - 'names.txt.2' saved [228145/228145]

